# Integrate melting curve of pure MgSiO3
* use direct clapeyron slope integration
* electronic contribution ignored due to bug in adiabatic calculation

In [ ]:
%run core.ipynb

analysis = load_analysis()
liq_eos_hybrid = analysis['liq_eos_hybrid']
liq_eos_S11 = analysis['liq_eos_S11']
liq_eos_dK09 = analysis['liq_eos_dK09']
sol_eos = analysis['sol_eos']

P_fus0 = analysis['P_fus0']
T_fus0 = analysis['T_fus0']
dS_fus_kB = analysis['dS_fus0']
S_fus0 = analysis['S_fus0']


In [ ]:
import matplotlib
matplotlib.get_configdir()
# plt.style.available

In [ ]:
# import matplotlib
matplotlib.get_configdir()
plt.style.use('presentation')
# plt.style.use('seaborn-talk')


In [ ]:
# liq_eos_S11.apply_electronic = False
# liq_eos_dK09.apply_electronic = False
# 
# liq_eos_S11.apply_electronic = True
# liq_eos_dK09.apply_electronic = True

In [ ]:
# P_fus0 = 23;
# T_fus0_adj = 2500;

P_fus0 = 25;
T_fus0_adj = 2700;

#T_fus0 = 2900;
#T_fus0 = 2700;
print(P_fus0, ' GPa, ',  T_fus0_adj, ' K')

In [ ]:
# liq_eos_hybrid0 = liq_eos_hybrid
# liq_compress_eos = liq_eos_hybrid0.compress_eos
# liq_thermal_eos = liq_eos_hybrid0.thermal_eos

In [ ]:
# liq_eos_hybrid = HybridEos(liq_compress_eos, liq_thermal_eos, 
#                        PTS_fus0=(P_fus0, T_fus0, S_fus0), fix_ref_adiabat=True)

In [ ]:
from scipy import interpolate
def fun_deriv(y, x, fac=0.01):
    dx = fac*np.median(np.diff(x))
    fun = interpolate.interp1d(x, y, kind='cubic', fill_value='extrapolate')
    df = fun(x+.5*dx)-fun(x-0.5*dx)
    dfdx = df/dx
    return dfdx

In [ ]:

PT_sol_Stx09 = pd.read_csv('data/Stixrude2009-solidus.csv')
PT_sol_Hmn13 = pd.read_csv('data/Hamano2013-solidus.csv')

In [ ]:
PT_liq_Stx09 = pd.read_csv('data/Stixrude2009-liquidus.csv')
PT_liq_Hmn13 = pd.read_csv('data/Hamano2013-liquidus.csv')
PT_liq_And11 = pd.read_csv('data/Andrault2011-liquidus.csv')

## Calculate melting curve

In [ ]:
def melting_curve_deriv(P, TVlVs, liq_eos, sol_eos, dS_fus_kB=None, 
                        dS_adj=0):
    PV_ratio = xmeos.models.CONSTS['PV_ratio']
    T, Vliq, Vsol = TVlVs[0], TVlVs[1], TVlVs[2]
    # print('P', P, ', T', T, ', Vliq', Vliq, ', Vsol',Vsol)
    
    if dS_fus_kB is None:
        dS = liq_eos.entropy(Vliq, T) - sol_eos.entropy(Vsol, T)
    else:
        dS = dS_fus_kB*xmeos.models.CONSTS['kboltz']
        
    dS = dS + dS_adj*xmeos.models.CONSTS['kboltz']
    # print('T = ',T,'   dS = ', dS)
    dV = Vliq - Vsol
     
    
    alpha_sol = sol_eos.thermal_exp(Vsol, T)
    alpha_liq = liq_eos.thermal_exp(Vliq, T)
    
    KT_sol = sol_eos.bulk_mod(Vsol, T)
    KT_liq = liq_eos.bulk_mod(Vliq, T)
    
    dTdP = dV/dS/PV_ratio
    # print('  dTdP = ', dTdP)
    dVdP_sol = Vsol*(alpha_sol*dTdP - 1.0/KT_sol)
    dVdP_liq = Vliq*(alpha_liq*dTdP - 1.0/KT_liq)
    TVV_deriv = np.hstack((dTdP, dVdP_liq, dVdP_sol))
    return TVV_deriv

In [ ]:

def calc_melting_curve(Pinit, Tinit, liq_eos, sol_eos, 
                       Pmax=150.0, Pmin=0.0, N=101, Vinit=9.0, 
                       dS_adj=0,
                       dS_fus_kB=None):
    Vinit_liq = volume(Pinit, Tinit, liq_eos, Vinit=Vinit)
    Vinit_sol = volume(Pinit, Tinit, sol_eos)
    
    # print(liq_eos.press(Vinit_liq, Tinit))
    # print(sol_eos.press(Vinit_sol, Tinit))
    
    deriv_fun = (lambda P, TVlVs, liq_eos=liq_eos, sol_eos=sol_eos, 
                 dS_fus_kB=dS_fus_kB: (melting_curve_deriv(
                     P, TVlVs, liq_eos, sol_eos, dS_fus_kB=dS_fus_kB, 
                     dS_adj=dS_adj) ))
    
    # Vinit = volume(Pinit, Tinit, eos, Vinit=Vinit_guess)
    # Vinit = eos.volume(Pinit, Tinit, Vinit_guess)
    def zero_temp(t, y): 
        T, Vliq, Vsol = y[0], y[1], y[2]
        dS = liq_eos.entropy(Vliq, T) - sol_eos.entropy(Vsol, T)
        if np.isnan(T):
            T = 0
            
        return T
    
    zero_temp.terminal = True
    zero_temp.direction = -1
    
    def zero_dS_fus(t, y, liq_eos=liq_eos, sol_eos=sol_eos): 
        T, Vliq, Vsol = y[0], y[1], y[2]
        dS = liq_eos.entropy(Vliq, T) - sol_eos.entropy(Vsol, T)
        if np.isnan(dS):
            dS = 0
        
        return dS
        
    
    zero_dS_fus.terminal = True
    zero_dS_fus.direction = -1
    
    
    P_path = np.linspace(Pinit, Pmax, N)
    output = sp.integrate.solve_ivp(deriv_fun, P_path[[0,-1]] , 
                                    np.array([Tinit, Vinit_liq, Vinit_sol]), 
                                    t_eval=P_path, vectorized=True, events=[zero_temp, zero_dS_fus])
    T_fus_pos, V_fus_liq_pos, V_fus_sol_pos = (
        output['y'][0], output['y'][1], output['y'][2])
    P_path = output.t
    
    
    P_path_neg = np.linspace(P_fus0, Pmin, N)
    output_neg = sp.integrate.solve_ivp(deriv_fun, P_path_neg[[0,-1]] , 
                                    np.array([Tinit, Vinit_liq, Vinit_sol]), 
                                    t_eval=P_path_neg, vectorized=True, events=[zero_temp, zero_dS_fus])
    T_fus_neg, V_fus_liq_neg, V_fus_sol_neg = (
        output_neg['y'][0], output_neg['y'][1], output_neg['y'][2])
    
    P_path_neg = output_neg.t
    
    # P_fus = np.hstack((P_path))
    # T_fus = np.hstack((T_fus_pos))
    # V_fus_liq = np.hstack((V_fus_liq_pos))
    # V_fus_sol = np.hstack((V_fus_sol_pos))
    
    P_fus = np.hstack((P_path_neg[::-1][:-1], P_path))
    T_fus = np.hstack((T_fus_neg[::-1][:-1], T_fus_pos))
    V_fus_liq = np.hstack((V_fus_liq_neg[::-1][:-1], V_fus_liq_pos))
    V_fus_sol = np.hstack((V_fus_sol_neg[::-1][:-1], V_fus_sol_pos))
    
    return P_fus, T_fus, V_fus_liq, V_fus_sol
    

In [ ]:
.16-.058

In [ ]:
dS_adj = 0.058
dS_adj = 0.16


Pmin=20
# Pmax = 136
Pmax = 150

In [ ]:


P_fus_S11, T_fus_S11, V_fus_liq_S11, V_fus_sol_S11 = (
    calc_melting_curve(P_fus0, T_fus0_adj, liq_eos_S11, sol_eos, 
                       Pmin=Pmin, Pmax=Pmax,  dS_adj=dS_adj))
P_fus_dK09, T_fus_dK09, V_fus_liq_dK09, V_fus_sol_dK09 = (
    calc_melting_curve(P_fus0, T_fus0_adj, liq_eos_dK09, sol_eos,
                       Pmin=Pmin, Pmax=Pmax, dS_adj=dS_adj))

P_fus_S11_fix, T_fus_S11_fix, V_fus_liq_S11_fix, V_fus_sol_S11_fix = (
    calc_melting_curve(P_fus0, T_fus0_adj, liq_eos_S11, sol_eos, 
                       dS_fus_kB=dS_fus_kB, Pmin=Pmin, Pmax=Pmax, 
                       dS_adj=dS_adj))
P_fus_dK09_fix, T_fus_dK09_fix, V_fus_liq_dK09_fix, V_fus_sol_dK09_fix = (
    calc_melting_curve(P_fus0, T_fus0_adj, liq_eos_dK09, sol_eos, 
                       dS_fus_kB=dS_fus_kB, Pmin=Pmin, Pmax=Pmax, 
                       dS_adj=dS_adj))



In [ ]:
# P_fus_hyb, T_fus_hyb, V_fus_liq_hyb, V_fus_sol_hyb = calc_melting_curve(
#     P_fus0, T_fus0_adj, liq_eos_hybrid, sol_eos, Pmin = 15)

In [ ]:

dVdP_sol = fun_deriv(V_fus_sol_dK09, P_fus_dK09)
dVdP_liq = fun_deriv(V_fus_liq_dK09, P_fus_dK09)

beta_fac = (dVdP_liq/dVdP_sol)/(V_fus_liq_dK09/V_fus_sol_dK09)

# plt.figure()
# plt.plot(P_fus_dK09, beta_fac,'k-')
# plt.ylim(1,3.5)
# 
# 
# plt.figure()
# plt.plot(P_fus_dK09, V_fus_sol_dK09,'k-')
# plt.plot(P_fus_dK09, V_fus_liq_dK09,'r-')


In [ ]:
# liq_eos_S11.press(V_fus_liq_S11, T_fus_S11)
# liq_eos_dK09.press(V_fus_liq_dK09, T_fus_dK09)
# sol_eos.press(V_fus_sol_S11, T_fus_S11)
# sol_eos.press(V_fus_sol_dK09, T_fus_dK09)

In [ ]:
# P_ad_hybrid, V_ad_hybrid, T_ad_hybrid = integrate_adiabat(
#     P_fus0, T_fus0, liq_eos_hybrid)

P_ad_dK09, V_ad_dK09, T_ad_dK09 = integrate_adiabat(
    P_fus0, T_fus0, liq_eos_dK09)

P_ad_S11, V_ad_S11, T_ad_S11 = integrate_adiabat(
    P_fus0, T_fus0, liq_eos_S11)

P_sol_ad, V_sol_ad, T_sol_ad = integrate_adiabat(
    P_fus0, T_fus0, sol_eos)

In [ ]:
plt.figure()
# plt.plot(P_fus_hyb, T_fus_hyb, 'k-', lw=2, label='Melt Curve (hybrid)')
plt.plot(P_fus_dK09, T_fus_dK09, 'r-', lw=2, label='Melt Curve (dK09)')
plt.plot(P_fus_S11, T_fus_S11, 'b-', lw=2, label='Melt Curve (S11)')

plt.plot(P_fus_dK09_fix, T_fus_dK09_fix, 'r:', lw=2, label='Melt Curve (dK09-fix)')
plt.plot(P_fus_S11_fix, T_fus_S11_fix, 'b:', lw=2, label='Melt Curve (S11-fix)')

plt.plot(PT_liq_Stx09['P'], PT_liq_Stx09['T'], 'r:', label='Melt Curve (Stixrude09)')
plt.plot(PT_liq_Hmn13['P'], PT_liq_Hmn13['T'], 'b:', label='Melt Curve (Hamano13)')
plt.plot(PT_liq_And11['P'], PT_liq_And11['T'], 'g:', label='Melt Curve (Andrault11)')

# plt.plot(P_ad_hybrid, T_ad_hybrid, 'k--', lw=3, label='Adiabat (hybrid)')
plt.plot(P_ad_dK09, T_ad_dK09, 'r--', label='Adiabat (dK09)')
plt.plot(P_ad_S11, T_ad_S11, 'b--', label='Adiabat (S11)')
plt.plot(P_sol_ad, T_sol_ad, 'g-.', label='Adiabat (solid)')


plt.title('MgSiO3 Melting curve predictions')
plt.xlabel('Pressure  [ GPa ]')
plt.ylabel('Temperature  [ K ]')
plt.legend()
plt.tight_layout()

In [ ]:
Pad_ref = 25
Tad_ref = 3450
Tad_ref = 3400
# Tad_ref = 3350
P_ad_dK09, V_ad_dK09, T_ad_dK09 = integrate_adiabat(
    Pad_ref, Tad_ref, liq_eos_dK09)

P_ad_S11, V_ad_S11, T_ad_S11 = integrate_adiabat(
    Pad_ref, Tad_ref, liq_eos_S11)

P_sol_ad, V_sol_ad, T_sol_ad = integrate_adiabat(
    Pad_ref, Tad_ref, sol_eos)

In [ ]:
plt.figure()
# plt.plot(P_fus_hyb, T_fus_hyb, 'k-', lw=2, label='Melt Curve (hybrid)')
plt.plot(P_fus_dK09, T_fus_dK09, 'k-', lw=2, label='This study')
# plt.plot(P_fus_S11, T_fus_S11, 'b-', lw=2, label='Melt Curve (S11)')

# plt.plot(P_fus_dK09_fix, T_fus_dK09_fix, 'k:', lw=2, label='This study (Sfus=1.5)') 
# plt.plot(P_fus_S11_fix, T_fus_S11_fix, 'b:', lw=2, label='Melt Curve (S11-fix)')

plt.plot(PT_liq_Stx09['P'], PT_liq_Stx09['T'], 'r:', label='Stixrude09')
plt.plot(PT_liq_Hmn13['P'], PT_liq_Hmn13['T'], 'g-', label='Melt Curve (Hamano13)')
plt.plot(PT_liq_And11['P'], PT_liq_And11['T'], 'b:', label='Andrault11')

# plt.plot(P_ad_hybrid, T_ad_hybrid, 'k--', lw=3, label='Adiabat (hybrid)')
# plt.plot(P_ad_dK09, T_ad_dK09, 'r-.', label='Liquid Adiabat (dK09)')
plt.plot(P_ad_dK09, T_ad_dK09, '--', color=[.5,.5,.5],label='Liquid Adiabat (Wolf2018)')
# plt.plot(P_ad_S11, T_ad_S11, 'b--', label='Adiabat (S11)')
# plt.plot(P_sol_ad, T_sol_ad, 'g-.', label='Adiabat (solid)')

plt.plot(PT_sol_Stx09['P'], PT_sol_Stx09['T'], 'b-', label='Stixrude09')
plt.plot(PT_sol_Hmn13['P'], PT_sol_Hmn13['T'], 'g-', label='Hamano13')


# plt.title('Rough Mantle Liquidus')
plt.xlabel('Pressure  [ GPa ]')
plt.ylabel('Temperature  [ K ]')
plt.legend()

plt.tight_layout()

plt.savefig('figs/magma-ocean-liquidus.png')

In [ ]:


S_fus_liq_S11 = liq_eos_S11.entropy(V_fus_liq_S11, T_fus_S11) 
S_fus_sol_S11 = sol_eos.entropy(V_fus_sol_S11, T_fus_S11) 
dS_fus_S11 = (S_fus_liq_S11-S_fus_sol_S11)/xmeos.models.CONSTS['kboltz']

S_fus_liq_dK09 = liq_eos_dK09.entropy(V_fus_liq_dK09, T_fus_dK09) 
S_fus_sol_dK09 = sol_eos.entropy(V_fus_sol_dK09, T_fus_dK09)
dS_fus_dK09 = (S_fus_liq_dK09-S_fus_sol_dK09)/xmeos.models.CONSTS['kboltz']

# S_fus_liq_hyb = liq_eos_hybrid.entropy(V_fus_liq_hyb, T_fus_hyb) 
# S_fus_sol_hyb = sol_eos.entropy(V_fus_sol_hyb, T_fus_hyb)
# dS_fus_hyb = (S_fus_liq_hyb-S_fus_sol_hyb)/xmeos.models.CONSTS['kboltz']


In [ ]:
mantle_liquidus = pd.DataFrame({'P':P_fus_dK09, 'T':T_fus_dK09, 'S/kB': S_fus_liq_dK09/xmeos.models.CONSTS['kboltz']})
T_solidus = T_fus_dK09/1.09
V_solidus = np.hstack([volume(iP, iT, liq_eos_dK09, Vinit=8) for (iP, iT) 
                       in zip(P_fus_dK09, T_solidus)])
S_fus_sol_dK09 = liq_eos_dK09.entropy(V_solidus, T_solidus) 

mantle_solidus = pd.DataFrame({'P':P_fus_dK09, 'T':T_solidus, 'S/kB': S_fus_sol_dK09/xmeos.models.CONSTS['kboltz']})

In [ ]:
# mantle_liquidus.to_csv('figs/mantle_liquidus_path.csv')
# mantle_solidus.to_csv('figs/mantle_solidus_path.csv')

In [ ]:
def write_output( file_name, data_tbl_a, data_units_a,
                 param_d, comment="" ):

    '''Write output.'''

    file_path = param_d['datadir']+file_name
    data_units = ' '.join([np.str(unit_val) for unit_val in data_units_a])
    header = param_d['header_default']+data_units
    size = '5 ' + str(int(data_tbl_a.shape[0])) + '\n'
    header = size + header
    if comment is not "":
        header = header + '\n' + comment
    np.savetxt( file_path, data_tbl_a, header=header)

#====================================================================
def write_data_table( file_name, data_cols_L, data_units_L,
                     param_d, comment="" ):

    '''Write data table with scale factors for each column.

    Input:
        file_name: string giving path to data file
        data_cols_L: list of arrays defining each column
        data_units_L: list of scale factors or string IDs (keys for param_d)
            referring to each scale factor
        param_d: dictionary containing output header and scale 
            factor info'''

    data_units_a = []
    for units in data_units_L:
        if isinstance( units, str ):
            data_units_a.append( param_d[units] )
        else:
            data_units_a.append( units )

    data_units_a = np.array( data_units_a )
    data_tbl_a = np.vstack( data_cols_L ).T
    write_output( file_name, data_tbl_a, data_units_a, param_d, comment=comment )
    
    
def get_output_constants(mass_avg, datadir):
    Nmol=6.0221413e+23
    output_d = OrderedDict()
    output_d['header_default'] = "Pressure, Entropy, Quantity\n"\
        "column * scaling factor should be SI units\n"\
        "scaling factors (constant) for each column given on line below\n"
    #output_d['datadir'] = 'data/lookup/lookup-rough/'
    #output_d['datadir'] = 'data/lookup/lookup-hires/'
    # output_d['datadir'] = 'data/lookup/lookup-hires-RTmelt/'
    output_d['datadir'] = datadir

    # NOTE: All extensive quantities changed from per atom to per unit mass
    # assert 'mass_avg' in param_d, "'mass_avg' must be set in param_d " \
    #     "in order to report extensive quantities per unit mass"
    # assert 'Nmol' in param_d, "'Nmol' is unset in param_d"

    # All output constants in mks units
    output_d['1'] = 1 # No unit change
    output_d['g'] = 1e-3 # mass [g] -> [kg]
    output_d['per_mass']  = Nmol/(mass_avg*output_d['g'])

    output_d['GPa'] = 1e9 # P:  [GPa] -> [Pa]
    output_d['GPa-1'] = 1e-9 # 1/P:  [1/GPa] -> [1/Pa]
    output_d['eV'] = 1.60217657e-19 \
        *output_d['per_mass'] # E, H, T*S, T*Cp:  [eV/atom] -> [J/kg]
    output_d['g_cc'] = 1e3 # rho:  [g/cc] -> [kg/m^3]
    return output_d


def write_all_data_tables(phasename, props, datadir='./'):
    """
    Write phase-specific data files
    """

    output = get_output_constants(props['molar_mass'], datadir)
    
    write_data_table('temperature_' + phasename + '.dat',
                     (props[key].ravel() for key in 
                      ('P', 'S', 'T')),
                     ('GPa', 'eV', 1), output)
    write_data_table('density_' + phasename + '.dat',
                     (props[key].ravel() for key in 
                      ('P', 'S', 'rho')),
                     ('GPa', 'eV','g_cc'), output)
    write_data_table('heat_capacity_' + phasename + '.dat',
                     (props[key].ravel() for key in 
                      ('P', 'S', 'C_P')),
                     ('GPa','eV','eV'), output)
    write_data_table('thermal_exp_' + phasename + '.dat',
                     (props[key].ravel() for key in 
                      ('P', 'S', 'alpha')),
                     ('GPa','eV',1), output)
    write_data_table('adiabat_temp_grad_'+phasename+'.dat',
                     (props[key].ravel() for key in 
                      ('P', 'S', 'dTdP_S')),
                     ('GPa','eV','GPa-1'), output)
    pass

In [ ]:
H13_liq = pd.read_csv('data/Hamano2013-liquidus.csv')
H13_sol = pd.read_csv('data/Hamano2013-solidus.csv')

S09_liq = pd.read_csv('data/Stixrude2009-liquidus.csv')
S09_sol = pd.read_csv('data/Stixrude2009-solidus.csv')


In [ ]:
def overwrite_zero_val(table):
    y0 = sp.interpolate.interp1d(table['P'], table['T'], fill_value='extrapolate')(0)
    table['P'][0] = 0.0
    table['T'][0] = y0
    
overwrite_zero_val(H13_liq)
overwrite_zero_val(H13_sol)
overwrite_zero_val(S09_liq)
overwrite_zero_val(S09_sol)

In [ ]:
# mantle_liquidus = pd.DataFrame({'P':P_fus_dK09, 'T':T_fus_dK09, 'S/kB': S_fus_liq_dK09/xmeos.models.CONSTS['kboltz']})
# mantle_solidus = pd.DataFrame({'P':P_fus_dK09, 'T':T_solidus, 'S/kB': S_fus_sol_dK09/xmeos.models.CONSTS['kboltz']})

Pgrid = P_fus_dK09
T_liq = T_fus_dK09
T_sol = T_solidus

In [ ]:
Pgrid

In [ ]:
# def join_curves(x_low, y_low, x_hi, y_hi, x_thresh):
#     x_tot = np.hstack([x_low[x_low < x_thresh], 
#                        x_hi[x_hi >= x_thresh]])
#     y_tot = np.hstack([y_low[x_low < x_thresh], 
#                        y_hi[x_hi >= x_thresh]])
#     return x_tot, y_tot


def bounded_curves(xgrids, ygrids, thresh_vals):
    thresh_vals = np.array(thresh_vals)
    if len(thresh_vals.shape)==0:
        thresh_vals = np.array([thresh_vals])
        
    thresh_vals = np.hstack([-np.infty, thresh_vals, np.infty])
        
    x_tot = []
    y_tot = []
    for ind in np.arange(len(thresh_vals)-1):
        xbnds = [thresh_vals[ind], thresh_vals[ind+1]]
        x, y = xgrids[ind], ygrids[ind]
        
        mask = (x >= xbnds[0]) & (x < xbnds[1])
        # x_tot[ind] = x[mask]
        # y_tot[ind] = y[mask]
        
        x_tot.append(x[mask])
        y_tot.append(y[mask])
        
    x_tot = np.hstack(x_tot)
    y_tot = np.hstack(y_tot)
    
    return x_tot, y_tot


# Plow_liq, Tlow_liq, Plow_sol, Tlow_sol = (H13_liq['P'], H13_liq['T'], 
#                                           H13_sol['P'], H13_sol['T'])


Pth_liq = 23.0
Pth_sol = 24.2

Plow_liq, Tlow_liq, Plow_sol, Tlow_sol = (S09_liq['P'], S09_liq['T'], 
                                          S09_sol['P'], S09_sol['T'])


# Pmantle_liq, Tmantle_liq = join_curves(Plow_liq, Tlow_liq, Pgrid, T_liq, Pth_liq)
# Pmantle_sol, Tmantle_sol = join_curves(Plow_sol, Tlow_sol, Pgrid, T_sol, Pth_sol)


Pmantle_liq, Tmantle_liq = bounded_curves([Plow_liq, Pgrid], [Tlow_liq, T_liq], Pth_liq)
Pmantle_sol, Tmantle_sol = bounded_curves([Plow_sol, Pgrid], [Tlow_sol, T_sol], Pth_sol)

    

# Pmantle_liq = np.hstack([H13_liq['P'][H13_liq['P']<Pth_liq], 
#                          Pgrid[Pgrid>Pth_liq]])
# Tmantle_liq = np.hstack([H13_liq['T'][H13_liq['P']<Pth_liq], 
#                          T_liq[Pgrid>Pth_liq]])
# Pmantle_sol = np.hstack([H13_sol['P'][H13_sol['P']<Pth_sol], 
#                          Pgrid[Pgrid>Pth_sol]])
# Tmantle_sol = np.hstack([H13_sol['T'][H13_sol['P']<Pth_sol], 
#                          T_sol[Pgrid>Pth_sol]])

plt.figure()

plt.plot(Pgrid, T_liq, 'm-')
plt.plot(Plow_liq, Tlow_liq, 'c-')


plt.plot(Pgrid, T_sol, 'm-')
plt.plot(Plow_sol, Tlow_sol, 'c-')

plt.plot(Pmantle_liq, Tmantle_liq, 'r--',
        Pmantle_sol, Tmantle_sol, 'b--', lw=1)

plt.xlim(0, 150)
plt.ylim(1500,5000)
plt.xlabel('P')
plt.ylabel('T')

In [ ]:
def poly_smooth_curve(xbnds, fun_left, fun_right, coef4=0.0, coef5=0.0):
    
    
    xfrac=.01
    dx = 0.5*xfrac*np.diff(xbnds)
    x0, x1 = xbnds[:]
    
    # remap x to [-1,1] range
    
    y0 = fun_left(x0)
    y1 = fun_right(x1)
    
    
    
    # dydx0 = (y0-fun_left(x0-dx))/dx
    # dydx1 = (y1-fun_right(x1-dx))/dx
    dydx0 = (y0-fun_left(x0-dx))/xfrac
    dydx1 = (y1-fun_right(x1-dx))/xfrac
    
    # yfit = np.array([y0, dydx0, y1, dydx1])
    # Mfit = np.array([
    #     [x0**3, x0**2, x0, 1.0],
    #     [3*x0**2, 2*x0, 1.0, 0.0],
    #     [x1**3, x1**2, x1, 1.0],
    #     [3*x1**2, 2*x1, 1.0, 0.0]
    # ])
    
    # Mfit = np.array([
    #     [-1, +1, -1, 1],
    #     [3*(+1), 2*(-1), 1, 0],
    #     [+1, +1, +1, 1],
    #     [3*(+1), 2*(+1), 1, 0]
    # ])
    
    yfit = np.array([y0, dydx0, y1, dydx1, coef4, coef5])
    Mfit = np.array([
        [-1, +1, -1, +1, -1, 1],
        [5*(+1), 4*(-1), 3*(+1), 2*(-1), 1, 0],
        [+1, +1, +1, +1, +1, 1],
        [5*(+1), 4*(+1), 3*(+1), 2*(+1), 1, 0],
        [0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0]
    ])
    
    # print(Mfit)
    
    # coef = np.linalg.solve(Mfit, yfit)
    output = sp.linalg.lstsq(Mfit, yfit)
    coef = output[0]
    
    def fun(x, xbnds=xbnds, coef=coef):
        xrange = np.diff(xbnds)
        xfrac = (x-xbnds[0])/xrange
        xscl = 2*xfrac - 1
        y = np.polyval(coef, xscl)
        return y
    
    return fun, coef

def constrain_poly_smooth_curve(xbnds, fun_left, fun_right, order=5):
    
    
    xfrac=.01
    dx = 0.5*xfrac*np.diff(xbnds)
    x0, x1 = xbnds[:]
    
    # remap x to [-1,1] range
    
    y0 = fun_left(x0)
    y1 = fun_right(x1)
    
    
    
    # dydx0 = (y0-fun_left(x0-dx))/dx
    # dydx1 = (y1-fun_right(x1-dx))/dx
    dydx0 = (y0-fun_left(x0-dx))/xfrac
    dydx1 = (y1-fun_right(x1-dx))/xfrac
    
    # yfit = np.array([y0, dydx0, y1, dydx1])
    # Mfit = np.array([
    #     [x0**3, x0**2, x0, 1.0],
    #     [3*x0**2, 2*x0, 1.0, 0.0],
    #     [x1**3, x1**2, x1, 1.0],
    #     [3*x1**2, 2*x1, 1.0, 0.0]
    # ])
    
    # Mfit = np.array([
    #     [-1, +1, -1, 1],
    #     [3*(+1), 2*(-1), 1, 0],
    #     [+1, +1, +1, 1],
    #     [3*(+1), 2*(+1), 1, 0]
    # ])
    
    coef_exp = np.arange(order,0)
    -np.ones(order+1)
    
    yfit = np.array([y0, dydx0, y1, dydx1, coef4, coef5])
    Mfit = np.array([
        [-1, +1, -1, +1, -1, 1],
        [5*(+1), 4*(-1), 3*(+1), 2*(-1), 1, 0],
        [+1, +1, +1, +1, +1, 1],
        [5*(+1), 4*(+1), 3*(+1), 2*(+1), 1, 0],
        [0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0]
    ])
    
    # print(Mfit)
    
    # coef = np.linalg.solve(Mfit, yfit)
    output = sp.linalg.lstsq(Mfit, yfit)
    coef = output[0]
    
    def fun(x, xbnds=xbnds, coef=coef):
        xrange = np.diff(xbnds)
        xfrac = (x-xbnds[0])/xrange
        xscl = 2*xfrac - 1
        y = np.polyval(coef, xscl)
        return y
    
    return fun, coef

In [ ]:
# Smoothing bounds leaving Trans Zone intact
# Pbnds_smth_liq = [11.5, 23.0]
# Pbnds_smth_sol = [21.5, 25.5]

# Smoothing bounds masking transition zone/upper mantle variability in entropy
# Smoothing bounds masking transition zone/upper mantle variability in entropy
Pbnds_smth_liq = [4.0, 23.0]
# Pbnds_smth_sol = [7.0, 30.0]
Pbnds_smth_sol = [4.0, 28.0]
# coef4=-120, coef5=+55
# 8, -10, +20

In [ ]:

Pbnds_smth_liq = [6.0, 26.0]
Pbnds_smth_sol = [6.0, 30.0]

fun_liq_loP = sp.interpolate.interp1d(Plow_liq, Tlow_liq, 
                                      kind='cubic', fill_value='extrapolate' )
fun_liq_hiP = sp.interpolate.interp1d(Pgrid, T_liq,
                                      kind='cubic', fill_value='extrapolate' )


fun_liq, coef_liq = poly_smooth_curve(Pbnds_smth_liq, fun_liq_loP, fun_liq_hiP)
coef_liq




fun_sol_loP = sp.interpolate.interp1d(Plow_sol, Tlow_sol, 
                                      kind='cubic', fill_value='extrapolate' )
fun_sol_hiP = sp.interpolate.interp1d(Pgrid, T_sol,
                                      kind='cubic', fill_value='extrapolate' )

fun_sol, coef_sol = poly_smooth_curve(Pbnds_smth_sol, fun_sol_loP, fun_sol_hiP, coef4=-135, coef5=+68)
coef_sol

In [ ]:
Pmod_liq = np.linspace(*Pbnds_smth_liq, 101)
Pmod_sol = np.linspace(*Pbnds_smth_sol, 101)
plt.figure()
plt.plot(Pmantle_liq, Tmantle_liq, 'r-',
        Pmantle_sol, Tmantle_sol, 'b-')
plt.plot(Pmod_liq, fun_liq(Pmod_liq), 'c-', lw=1)
plt.plot(Pmod_sol, fun_sol(Pmod_sol), 'c-', lw=1)
plt.xlim(0, 150)
plt.ylim(1500,5000)
plt.xlabel('P')
plt.ylabel('T')

In [ ]:
def eval_vol(Pvals, Tvals, mod_eos, Vinit=10.0):
    Vvals = np.zeros(Pvals.size)
    iV = Vinit
    for ind, (iP, iT) in enumerate(zip(Pvals, Tvals)):
        iV = volume(iP, iT, mod_eos, Vinit=iV)
        Vvals[ind] = iV
        
    return Vvals, Pvals, Tvals

In [ ]:
Pmantle_liq_adj, Tmantle_liq_adj = bounded_curves(
    [Pmantle_liq, Pmod_liq, Pmantle_liq], 
    [Tmantle_liq, fun_liq(Pmod_liq), Tmantle_liq],
    [Pmod_liq[0], Pmod_liq[-1]])

Pmantle_sol_adj, Tmantle_sol_adj = bounded_curves(
    [Pmantle_sol, Pmod_sol, Pmantle_sol], 
    [Tmantle_sol, fun_sol(Pmod_sol), Tmantle_sol],
    [Pmod_sol[0], Pmod_sol[-1]])

fun_liq_adj = sp.interpolate.interp1d(Pmantle_liq_adj, Tmantle_liq_adj)
fun_sol_adj = sp.interpolate.interp1d(Pmantle_sol_adj, Tmantle_sol_adj)


In [ ]:
Pvals = np.arange(0,150,1)
# Pvals = np.arange(0,50,.5)
Tvals_liq = fun_liq_adj(Pvals)
Tvals_sol = fun_sol_adj(Pvals)

In [ ]:
Vsol, Psol, Tsol = eval_vol(Pvals, Tvals_sol, sol_eos, Vinit=9.0)
Vliq, Pliq, Tliq = eval_vol(Pvals, Tvals_liq, liq_eos_dK09, Vinit=12.0)

Ssol = sol_eos.entropy(Vsol, Tsol)
Sliq = liq_eos_dK09.entropy(Vliq, Tliq)

In [ ]:

plt.figure()
plt.plot(Psol, Tsol, 'b-')
plt.plot(Pliq, Tliq, 'r-')
plt.xlabel('P')
plt.ylabel('T')

plt.figure()
plt.plot(Psol, Ssol/models.CONSTS['kboltz'], 'b-')
plt.plot(Pliq, Sliq/models.CONSTS['kboltz'], 'r-')
plt.xlabel('P')
plt.ylabel('S')

In [ ]:
datadir='./data_tables/dK09-elec-free/'
molar_mass = sol_eos.molar_mass
output = get_output_constants(molar_mass, datadir)
write_data_table('solidus_smth_perturb.dat', (Psol, Ssol), ('GPa', 'eV'), output)
write_data_table('liquidus_smth_perturb.dat', (Pliq, Sliq), ('GPa', 'eV'), output)

In [ ]:

Vliq_low, Pliq_low, Tliq_low = eval_vol(Plow_liq, Tlow_liq, liq_eos_dK09, Vinit=12.0)
Vliq_hi, Pliq_hi, Tliq_hi = eval_vol(Pgrid, T_liq, liq_eos_dK09, Vinit=12.0)

Vsol_low, Psol_low, Tsol_low = eval_vol(Plow_sol, Tlow_sol, sol_eos, Vinit=9.0)
Vsol_hi, Psol_hi, Tsol_hi = eval_vol(Pgrid, T_sol, sol_eos, Vinit=9.0)

Ssol_low = sol_eos.entropy(Vsol_low, Tsol_low)
Ssol_hi = sol_eos.entropy(Vsol_hi, Tsol_hi)

Sliq_low = liq_eos_dK09.entropy(Vliq_low, Tliq_low)
Sliq_hi = liq_eos_dK09.entropy(Vliq_hi, Tliq_hi)

    
    

In [ ]:
plt.figure()
plt.plot(Psol_low, Tsol_low, 'c-')
plt.plot(Psol_hi, Tsol_hi, 'm-')

plt.plot(Pliq_low, Tliq_low, 'c-')
plt.plot(Pliq_hi, Tliq_hi, 'm-')
plt.xlabel('P')
plt.ylabel('T')

plt.figure()
plt.plot(Psol_low, Ssol_low/models.CONSTS['kboltz'], 'c-')
plt.plot(Psol_hi, Ssol_hi/models.CONSTS['kboltz'], 'm-')

plt.plot(Pliq_low, Sliq_low/models.CONSTS['kboltz'], 'c-')
plt.plot(Pliq_hi, Sliq_hi/models.CONSTS['kboltz'], 'm-')
plt.xlabel('P')
plt.ylabel('S')



In [ ]:
plt.figure()
plt.plot(Psol, Tsol, 'b-')
plt.plot(Pliq, Tliq, 'r-')
plt.xlabel('P')
plt.ylabel('T')

plt.figure()
plt.plot(Psol, Ssol/models.CONSTS['kboltz'], 'b-')
plt.plot(Pliq, Sliq/models.CONSTS['kboltz'], 'r-')
plt.xlabel('P')
plt.ylabel('S')



In [ ]:
dV_fus_dK09 = V_fus_liq_dK09 -  V_fus_sol_dK09
dV_fus_dK09/V_fus_liq_dK09
plt.figure()
plt.plot(P_fus_dK09, 100*dV_fus_dK09/V_fus_liq_dK09, 'k-' )
plt.ylim(0,15)
plt.fill_between([0,150], [0.268, 0.268], color=[1,.5,.5])
plt.xlim(20,136)

plt.xlabel('Pressure [GPa]')
plt.ylabel('Volume of Fusion [%]')

plt.tight_layout()
plt.savefig('figs/vol_fusion.png')


In [ ]:
m_vib = (0.067-0.104)/(135-25)
b_vib = 0.104 - 25*m_vib
S_vib = m_vib*P_fus_dK09 + b_vib


S_mix = 0.058

In [ ]:

plt.figure()
plt.plot(P_fus_dK09, dS_fus_dK09+S_vib+S_mix, 'k-', label = 'This work')
plt.plot(P_fus_dK09, 1.5*np.ones(P_fus_dK09.size), 'r:', lw=2, label='Stixrude09')
plt.plot(P_fus_dK09, dS_fus_dK09, 'k--', lw=2, label='Brg fusion')
plt.plot(P_fus_dK09, S_vib, 'm--', lw=2, label = 'vibration')
plt.plot(P_fus_dK09, S_mix*np.ones(P_fus_dK09.size), 'c--', lw=2, label = 'mixing')
plt.ylim(0,2.5)
# plt.fill_between([0,150], [0.268, 0.268], color=[1,.5,.5])
plt.xlim(20,136)

plt.legend(loc='lower right')
plt.xlabel('Pressure [GPa]')
plt.ylabel('Entropy of Fusion [$k_B$/atom]')

plt.tight_layout()
plt.savefig('/Users/aswolf/entropy_fusion.png')


In [ ]:
P_liq_And = np.linspace(np.ceil(PT_liq_And11['P'][0]), np.floor(np.array(PT_liq_And11['P'])[-1]), 101)
coef_And = np.polyfit(PT_liq_And11['P'], PT_liq_And11['T'], 5)
T_liq_And = np.polyval(coef_And, P_liq_And)



In [ ]:
def extract_sol_liq_vols(P, T, sol_eos, dS=1.5):
    dTdP_fus = fun_deriv(T, P)
    
    Vfus_sol = np.hstack([volume(iP, iT, sol_eos, Vinit=8) for (iP, iT) in zip(P, T)])
    dVfus = dTdP_fus*dS*xmeos.models.CONSTS['kboltz']*xmeos.models.CONSTS['PV_ratio']
    Vfus_liq = dVfus+Vfus_sol
    
    return Vfus_sol, Vfus_liq

Vfus_sol_And, Vfus_liq_And = extract_sol_liq_vols(P_liq_And, T_liq_And, sol_eos, dS=1.5)
Vfus_sol_dK09, Vfus_liq_dK09 = extract_sol_liq_vols(P_fus_dK09, T_fus_dK09, sol_eos, dS=1.5)


In [ ]:
def compressibility_ratio(P, Vfus_sol, Vfus_liq):
    dVdP_sol = fun_deriv(Vfus_sol, P)
    dVdP_liq = fun_deriv(Vfus_liq, P)
    beta_fac = (dVdP_liq/dVdP_sol)/(Vfus_liq/Vfus_sol)
    return beta_fac

beta_fac_And = compressibility_ratio(P_liq_And, Vfus_sol_And, Vfus_liq_And)
beta_fac_dK09_approx = compressibility_ratio(P_fus_dK09, Vfus_sol_dK09, Vfus_liq_dK09)
    

In [ ]:
# dS = 1.5*xmeos.models.CONSTS['kboltz']
# 
# 
# dTdP_fus_dK09 = fun_deriv(T_fus_dK09, P_fus_dK09)
# dTdP_fus_And = fun_deriv(T_liq_And, P_liq_And)
# 
# # Vfus_sol_And = np.hstack([volume(iP, iT, sol_eos, Vinit=9) for (iP, iT) in zip(PT_liq_And11['P'], PT_liq_And11['T'])])
# 
# 
# Vfus_sol_And = np.hstack([volume(iP, iT, sol_eos, Vinit=8) for (iP, iT) in zip(P_liq_And, T_liq_And)])
# dVfus_And = dTdP_fus_And*dS*xmeos.models.CONSTS['PV_ratio']
# Vfus_liq_And = dVfus_And+Vfus_sol_And
# 
# 
# # dVdP_sol_And = fun_deriv(Vfus_sol_And, PT_liq_And11['P'])
# # dVdP_liq_And = fun_deriv(Vfus_liq_And, PT_liq_And11['P'])
# dVdP_sol_And = fun_deriv(Vfus_sol_And, P_liq_And)
# dVdP_liq_And = fun_deriv(Vfus_liq_And, P_liq_And)
# beta_fac_And = (dVdP_liq_And/dVdP_sol_And)/(Vfus_liq_And/Vfus_sol_And)


In [ ]:


# plt.figure()
# plt.plot(P_fus_dK09, dS*dTdP_fus_dK09*xmeos.models.CONSTS['PV_ratio'], 'k-')
# plt.plot(P_liq_And, dS*dTdP_fus_And*xmeos.models.CONSTS['PV_ratio'], 'k--')




plt.figure()
plt.plot(P_fus_dK09, beta_fac,'k-', label='This study')
plt.plot(P_fus_dK09, beta_fac_dK09_approx,'r--', label='Stixrude09')
plt.plot(P_liq_And, beta_fac_And,'b--', label='Andrault11')
plt.xlabel('Pressure  [GPa]')
plt.ylabel('Compressibility Ratio')
plt.legend()


plt.fill_between(P_liq_And,0, 1, color=[1,.8,.8])
plt.ylim(.5,3.5)
plt.xlim(15,140)
plt.tight_layout()

plt.savefig('/Users/aswolf/compressibility_ratio.png')


In [ ]:


# plt.figure()
# plt.plot(P_fus_dK09, dS*dTdP_fus_dK09*xmeos.models.CONSTS['PV_ratio'], 'k-')
# plt.plot(P_liq_And, dS*dTdP_fus_And*xmeos.models.CONSTS['PV_ratio'], 'k--')




plt.figure()
plt.plot(Vfus_liq_dK09/Vfus_sol_dK09-1, beta_fac,'k-')
plt.plot(Vfus_liq_dK09/Vfus_sol_dK09-1, beta_fac_dK09_approx,'k--')
plt.plot(Vfus_liq_And/Vfus_sol_And-1, beta_fac_And,'b--')
plt.ylim(.5,3.5)
#plt.xlim(15,140)


In [ ]:
plt.figure()
dTdP_ad_dK09 = fun_deriv(T_ad_dK09, P_ad_dK09)
dTdP_fus_dK09 = fun_deriv(T_fus_dK09, P_fus_dK09)
err_env = 0.268/100*V_fus_liq_dK09/dV_fus_dK09


plt.plot(P_ad_dK09, dTdP_ad_dK09, '--', color=[.5,.5,.5], label='Liquid Adiabat (dK09)')

plt.plot(P_fus_dK09, dTdP_fus_dK09, 'r-', label='This Study', lw=1)
plt.fill_between(P_fus_dK09, (1+err_env)*dTdP_fus_dK09, (1-err_env)*dTdP_fus_dK09, color=[1,.7,.7] )
plt.xlabel('Pressure [GPa]')
plt.ylabel('$dT/dP$   [K/GPa]')
plt.legend()
# plt.plot(P_fus_dK09, (1+err_env)*dTdP_fus_dK09, 'k--', label='dK09')
# plt.plot(P_fus_dK09, (1-err_env)*dTdP_fus_dK09, 'k--', label='dK09')

plt.ylim(0, 60)
plt.xlim(20, 140)


a = plt.axes([.55, .45, .35, .35])
plt.plot(P_ad_dK09, dTdP_ad_dK09, '--', color=[.5,.5,.5], label='Liquid Adiabat (dK09)')
plt.plot(P_fus_dK09, dTdP_fus_dK09, 'r-', label='This Study', lw=1)
plt.fill_between(P_fus_dK09, (1+err_env)*dTdP_fus_dK09, (1-err_env)*dTdP_fus_dK09, color=[1,.7,.7] )
plt.plot([101.5,101.5],[0,8.9],'k:',lw=1)
plt.plot([94.5,94.5],[0,9.3],'k:',lw=1)
plt.plot([108.5,108.5],[0,8.7],'k:',lw=1)
# plt.xticks([95,100,110])

plt.xlim(90,115)
plt.ylim(8,10.5)
plt.tight_layout()
plt.savefig('/Users/aswolf/dTdP_liquidus.png')

In [ ]:
101.5 +/- 7

In [ ]:

plt.figure()

plt.plot(P_fus_dK09, dS_fus_dK09, 'r--')
plt.plot(P_fus_S11, dS_fus_S11, 'b--')
# plt.plot(P_fus_hyb, dS_fus_hyb, 'k--')

plt.xlabel('Pressure  [ GPa ]')
plt.ylabel('dS_fus  [ $k_B$ ]')


In [ ]:

plt.figure()

plt.plot(P_fus_dK09, 1.5/dS_fus_dK09, 'r--')
# plt.plot(P_fus_hyb, dS_fus_hyb, 'k--')

plt.xlabel('Pressure  [ GPa ]')
plt.ylabel('1/dS_fus  [ $k_B$ ]')
plt.ylim(0,1.8)


In [ ]:
ind_ref = 100
P_fus_dK09[ind_ref]

In [ ]:
print('dS [kboltz/atom] = ', dS_fus_dK09[ind_ref])
print('dV [ang^3/atom] = ', dV_fus_dK09[ind_ref])

In [ ]:

plt.figure()
plt.plot(P_fus_dK09, dS_fus_dK09[ind_ref]/dV_fus_dK09[ind_ref]*
         dV_fus_dK09/dS_fus_dK09, 'k-', label='dV/dS')
plt.plot(P_fus_dK09, dV_fus_dK09/dV_fus_dK09[ind_ref], 'r--', label='dV')
plt.plot(P_fus_dK09, dS_fus_dK09[ind_ref]/dS_fus_dK09, 'b--', label='1/dS')

plt.xlabel('Pressure [GPa]')
plt.ylabel('Fractional Impact on Liquidus Slope')
plt.ylim(0,1.5)
plt.legend()

plt.savefig('figs/MgSiO3-liquidus-slope-dependence.png', dpi=450)